In [1]:
import os
import cv2
import numpy as np

# Function to load images from a directory and return them as a list
def load_images_from_folder(folder_path):
    images = []
    labels = []
    for label_folder in os.listdir(folder_path):
        label_folder_path = os.path.join(folder_path, label_folder)
        for filename in os.listdir(label_folder_path):
            img_path = os.path.join(label_folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
                labels.append(label_folder)  # Label can be 'car', 'motorcycle', or 'no_vehicle'
    return images, labels

# Load dataset
dataset_folder = '../dataset/images'
images, labels = load_images_from_folder(dataset_folder)
print(f"Loaded {len(images)} images with corresponding labels.")


Loaded 8110 images with corresponding labels.


In [2]:
from skimage.feature import hog

# Function to extract color histogram features
def extract_color_histogram(image, bins=32):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist_features = []
    for i in range(3):  # 3 channels in HSV
        hist = np.histogram(hsv_image[:, :, i], bins=bins, range=(0, 256))[0]
        hist_features.append(hist)
    return np.concatenate(hist_features)

# Function to extract spatial binning features
def extract_spatial_binning(image, size=(32, 32)):
    return cv2.resize(image, size).ravel()

# Function to extract HOG features with a size check
def extract_hog_features(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Ensure the image size is large enough for the given HOG parameters
    if gray_image.shape[0] >= pixels_per_cell[0] * cells_per_block[0] and gray_image.shape[1] >= pixels_per_cell[1] * cells_per_block[1]:
        features, _ = hog(gray_image, orientations=orientations, pixels_per_cell=pixels_per_cell,
                          cells_per_block=cells_per_block, visualize=True)
        return features
    else:
        # Return a placeholder if the image is too small for HOG
        print(f"Image is too small for HOG feature extraction. Size: {gray_image.shape}")
        return np.zeros(orientations * cells_per_block[0] * cells_per_block[1])

# Function to extract combined features (color histogram, spatial binning, and HOG)
def extract_combined_features(image):
        
    color_features = extract_color_histogram(image, bins=32)
    spatial_features = extract_spatial_binning(image, size=(32, 32))
    hog_features = extract_hog_features(image)  # Extract HOG features with size check
    return np.concatenate((color_features, spatial_features, hog_features))

# Extract features from the dataset
features = [extract_combined_features(img) for img in images]


Image is too small for HOG feature extraction. Size: (14, 18)
Image is too small for HOG feature extraction. Size: (31, 12)


KeyboardInterrupt: 

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode labels into numerical values
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)


In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

# Discretize the features using KBinsDiscretizer
discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
discretized_features = discretizer.fit_transform(features)


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(discretized_features, encoded_labels, test_size=0.2, random_state=42)

# Train a Gaussian Naive Bayes classifier
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Test the model and print the classification report
y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


In [ ]:
# Sliding window function for object localization
def sliding_window(image, step_size, window_size):
    for y in range(0, image.shape[0] - window_size[1], step_size):
        for x in range(0, image.shape[1] - window_size[0], step_size):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])

# Function to detect vehicles in a video frame using sliding window and classifier
def detect_vehicles_in_frame(frame, model, label_encoder, window_size=(128, 128), step_size=32):
    detected_objects = []
    for (x, y, window) in sliding_window(frame, step_size, window_size):
        if window.shape[0] != window_size[1] or window.shape[1] != window_size[0]:
            continue
        
        # Extract features for the current window
        window_features = extract_combined_features(window).reshape(1, -1)
        
        # Discretize the features for Naive Bayes
        window_features = discretizer.transform(window_features)
        
        # Predict the class using the trained model
        prediction = model.predict(window_features)
        predicted_class = label_encoder.inverse_transform(prediction)[0]
        
        if predicted_class != 'no_vehicle':
            detected_objects.append((x, y, predicted_class))
            cv2.rectangle(frame, (x, y), (x + window_size[0], y + window_size[1]), (0, 255, 0), 2)
            cv2.putText(frame, predicted_class, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return frame, detected_objects


In [ ]:
def process_video(input_path, output_path, model, label_encoder, window_size=(128, 128), step_size=32):
    cap = cv2.VideoCapture(input_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'DIVX'), 20, (width, height))

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            # Detect vehicles in the current frame
            frame, detected_objects = detect_vehicles_in_frame(frame, model, label_encoder, window_size, step_size)
            writer.write(frame)
            frame_idx += 1
        else:
            break

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    writer.release()
    cv2.destroyAllWindows()
    print(f"Video processing complete. Output saved to {output_path}")

# Example usage
input_video_path = './dataset/sample_videos/sample_2.mp4'
output_video_path = './Prediction/Naive_bayes_vehicle_detection_output.mp4'

# Process the video
process_video(input_video_path, output_video_path, nb_model, label_encoder)
